In [1]:

from Experiment import Experiment
import matplotlib.pyplot as plt
from Plotters import Plotters
import plotly.graph_objects as go
import plotly
import numpy as np
import time
%matplotlib qt

loadir = 'H:/My Drive/dark 2022/csv_dark' # dir to save the data frame in # H:\My Drive\Ronis Exp\sagiv\data_analysis
exp_name = ['manipulated_2022_05_19_40ms']
# ,'manipulated_2023_08_09_80ms','manipulated_2023_08_06_40ms',
#             'manipulated_2023_08_10_100ms','manipulated_2022_02_03_dark','manipulated_2023_08_07_5ms','manipulated_2023_08_07_10ms']#,'manipulated_2022_02_03_dark','manipulated_2023_08_06_40ms','manipulated_2023_08_09_60ms','manipulated_2023_08_07_5ms','manipulated_2023_08_07_10ms']

ref_frame = 0
experiments = [Experiment(loadir,exp) for exp in exp_name]
min_max_v = [exp.min_max_point_movies('v_xy') for exp in experiments]
min_max_pitch =[exp.min_max_point_movies('pitch_y_frame') for exp in experiments]
zero_velocity = [exp.zero_velocity_movies('v_xy') for exp in experiments]
[exp.pqr_movies() for exp in experiments]
[exp.mean_mean_props_movies('phi_rw','phi_lw','mean_wing','mean_mean_phi') for exp in experiments]




[None]

In [4]:
import matplotlib.cm as colormap

exp = experiments[-1]
mov = exp.get_mov('mov1')
color_map = colormap.datad["tab10"]['listed']
fig = mov.plot_prop('v_xy','body',color_map[0],mov.name,prop_x = 'time',t0 = 100,t1 = 400)
fig.show()


TypeError: 'numpy.ndarray' object is not callable

In [4]:
import matplotlib.cm as colormap
fig = go.Figure()
exp = experiments[-1]

# exp.plot_movie_prop('time','v_xy', case = 'plot_exp',prop = 'body', 
#                      fig = fig, mov = False,mode = 'markers') 
# fig.show()
color_map = colormap.datad["tab10"]['listed']

exp.plot_prop_movies('v_xy','body',color_map[0],'wak',mov = False,case = 'plot_mov')



TypeError: 'numpy.ndarray' object is not callable

In [ ]:
import numpy as np

fig = go.Figure()
[exp.plot_movie_prop('time','CM_xy_dot', case = 'plot_exp',prop = 'body', fig = fig, mov = False,add_horizontal_line = 0, color = idx + 2) for idx,exp in enumerate(experiments)]
fig.show()


fig = go.Figure()
[exp.plot_movie_prop('time','pitch_y_frame', case = 'plot_exp',prop = 'body', 
                     fig = fig, mov = False,add_horizontal_line = 0, color = idx + 2) for idx,exp in enumerate(experiments)]
fig.show()



plt.figure()
delta_zero = [plt.hist(np.array(zero_velocity[idx]) ,alpha=0.5,bins = 20,label = exp.experiment_name) for idx,exp in enumerate(experiments) if exp.pertubation != False]
plt.title('time of zero velocity')
plt.legend()

plt.figure()
[plt.hist(np.array(min_max_v[idx][0]) ,alpha=0.5,bins = 20,label = exp.experiment_name) for idx,exp in enumerate(experiments) if exp.pertubation != False]
plt.legend()
plt.title('time of minimal velocity')


plt.figure()
[plt.hist(np.array(min_max_pitch[idx][1]),alpha=0.5,bins = 15,label = exp.experiment_name,range=(50, 120)) for idx,exp in enumerate(experiments) if exp.pertubation != False]
plt.legend()
plt.title('time of maximal pitch')


plt.figure()
[plt.hist(np.array(min_max_v[idx][0] - min_max_pitch[idx][1] ),alpha=0.5,bins = 20,label = exp.experiment_name) for idx,exp in enumerate(experiments[0:2]) if exp.pertubation != False]
plt.legend()
plt.title('diff max pitch min v')




Text(0.5, 1.0, 'diff max pitch min v')

mean of all lines. normalized: data - mean(data(0:t == 0))
velocity
pitch
mean mean phi

- difference in peek between pulses - velocity and pitch
- pitch - long time reaction - above 100 ms
- (mean(phi)L + mean(phi)R)/2 - visible peek at ~55
- peek around 45 pitch_dot
- peek around 60 - p 

maybe a better way to do it? 


In [ ]:
N = 15
dt = 1/16000
# prop = 'pitch_y_frame'
prop = 'CM_xy_dot_dot'
# prop = 'mean_mean_phi'
wing_body = 'mean_wing'
wing_body = 'body'
# prop = 'pitch_y_frame_dot'
# prop = 'p'

conv = False

plt.figure()
for exp,color in zip(experiments[:],['blue','red','green','pink','orange']):
    for mov in exp.mov_names:
        cm_dot = exp.get_mov(mov).get_prop(prop,wing_body) 
        cm_dot = cm_dot - np.mean(cm_dot[0:exp.get_mov(mov).ref_frame + 1])
        time = exp.get_mov(mov).get_prop('time',wing_body)

        # plt.plot(time,cm_dot,color,alpha = 0.2)


for exp,color in zip(experiments[:],['blue','red','green','pink','orange']):
    t_list = np.arange(-40,400,dt*1000)
    cmlist = np.full((len(exp.mov_names),len(t_list)),np.nan)
    for idx_mov,mov in enumerate(exp.mov_names):
        time = exp.get_mov(mov).get_prop('time',wing_body)
        cm_dot = exp.get_mov(mov).get_prop(prop,wing_body)
        cm_dot = cm_dot - np.mean(cm_dot[0:exp.get_mov(mov).ref_frame + 1])
        if conv == True:
            cm_dot = np.convolve(cm_dot, np.ones(N)/N, mode='same')



        for idx,t in enumerate(time): 
            cmlist[idx_mov,t_list == t] =  cm_dot[idx]

    
        
    plt.plot(t_list,np.nanmean(cmlist,axis = 0),color = color,linestyle='None',
             label = exp.experiment_name,linewidth = 5,marker = 'o')
    plt.hlines(0,t_list[0],t_list[-1],color = 'black')
plt.legend()




C:\Users\Roni\AppData\Local\Temp\ipykernel_36668\2206632149.py:40: RuntimeWarning:

Mean of empty slice



plot velocity, pitch and p per movie

In [ ]:
exp = experiments[2]
plt.figure()

for mov,color in zip(exp.mov_names[0:1],['red','blue','green','black']):
    mov = exp.get_mov(mov)
    cm_xy_dot_dot = mov.get_prop('CM_xy_dot_dot','body')/100
    cm_xy_dot = mov.get_prop('CM_xy_dot','body')
    time = mov.get_prop('time','body')

    pitch = mov.get_prop('pitch_y_frame','body')/100
    p = mov.get_prop('p','body')/9000

    plt.plot(time,cm_xy_dot_dot,color, linestyle = 'dashed')
    plt.plot(time,cm_xy_dot,color, linestyle = '--')

    plt.plot(time,pitch,color)
    plt.plot(time,p,color,linestyle=':')
    plt.hlines(0,0,time[-1],color = 'black')



In [ ]:
exp = experiments[0]

fig = go.Figure()
exp.plot_movie_prop('time','roll_body', case = 'plot_mov',prop = 'body', fig = fig, mov = False,add_horizontal_line = 0)
fig.show()

In [ ]:

exp = experiments[0]
mov = exp.get_mov('mov117')
mov.plot_3d_traj_movie('CM_xy_dot')

In [ ]:
exp = experiments[0]
mov = exp.get_mov('mov3')
mov.header['body']

{'pitch_body': 0,
 'yaw_body': 1,
 'roll_body': 2,
 'pitch_body_dot': 3,
 'yaw_body_dot': 4,
 'roll_body_dot': 5,
 'pitch_body_dot_dot': 6,
 'yaw_body_dot_dot': 7,
 'roll_body_dot_dot': 8,
 'frames': 9,
 'time': 10,
 'CM_real_x_body': 11,
 'CM_real_y_body': 12,
 'CM_real_z_body': 13,
 'phi_rw_max_idx': 14,
 'phi_rw_max_val': 15,
 'phi_lw_max_idx': 16,
 'phi_lw_max_val': 17,
 'phi_rw_min_idx': 18,
 'phi_rw_min_val': 19,
 'phi_lw_min_idx': 20,
 'phi_lw_min_val': 21,
 'CM_xy': 22,
 'CM_xy_dot': 23,
 'CM_xy_dot_dot': 24,
 'yaw_z_frame': 25,
 'pitch_y_frame': 26,
 'roll_x_frame': 27,
 'yaw_z_frame_dot': 28,
 'pitch_y_frame_dot': 29,
 'roll_x_frame_dot': 30,
 'yaw_z_frame_dot_dot': 31,
 'pitch_y_frame_dot_dot': 32,
 'roll_x_frame_dot_dot': 33,
 'mean_idx': 34,
 'p': 35,
 'q': 36,
 'r': 37,
 'p_dot': 38,
 'q_dot': 39,
 'r_dot': 40,
 'p_dot_dot': 41,
 'q_dot_dot': 42,
 'r_dot_dot': 43}

In [ ]:

fig = go.Figure()
exp.plot_movie_prop('time','p', case = 'plot_mov',prop = 'body', fig = fig, mov = False,add_horizontal_line = 0)
fig.show()


In [ ]:
import plotly.graph_objects as go
import plotly

exp = experiments[0]
exp.pqr_movies() 
exp.add_mean_prop_movies('q','body','mean_wing') 
exp.mean_mean_props_movies('phi_rw_min_val','phi_lw_min_val','mean_wing','mean_front_phi')
exp.mean_mean_props_movies('phi_rw','phi_lw','mean_wing','mean_mean_phi')

fig = go.Figure()
exp.plot_movie_prop('time','q', case = 'plot_mean',prop = 'mean_wing', fig = fig, mov = False,add_horizontal_line = 0)
fig.show()

fig = go.Figure()

exp.plot_movie_prop('time','p', case = 'plot_mov',prop = 'body', fig = fig, mov = False,add_horizontal_line = 0)
fig.show()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()    
[fig.add_trace(go.Histogram(x=np.hstack(min), name=exp.experiment_name, nbinsx=30)) for min,exp in zip(zero_velocity,experiments)]
fig.update_layout(title = 'time of minimum velocity')
fig.update_traces(opacity = 0.9)
fig.show()


Save plots to folders

In [ ]:
import plotly.graph_objects as go
import plotly



def plot_and_save_movie_prop(exp,yprop,wing_body,add_horizontal_line = False):
    
    fig = go.Figure()
    exp.plot_movie_prop('time',yprop, case = 'plot_mov',prop = wing_body, fig = fig, mov = False,add_horizontal_line = add_horizontal_line) 
    fig.show()
    plotly.offline.plot(fig, filename=f'{exp.figures_path}/{yprop}_plot.html',auto_open=False)




for exp in experiments:

    exp.plot_3d_traj_movies('CM_xy_dot',save_plot = True, mov = False)

    plot_and_save_movie_prop(exp,yprop = 'CM_xy_dot',wing_body = 'body',add_horizontal_line = 0)
    plot_and_save_movie_prop(exp,yprop = 'pitch_y_frame',wing_body = 'body',add_horizontal_line = 0)

    exp.mean_mean_props_movies('phi_rw_min_val','phi_lw_min_val','mean_wing','mean_front_phi')
    exp.mean_mean_props_movies('phi_rw','phi_lw','mean_wing','mean_mean_phi')

    plot_and_save_movie_prop(exp,yprop = 'mean_front_phi',wing_body = 'mean_wing')
    plot_and_save_movie_prop(exp,yprop = 'mean_mean_phi',wing_body = 'mean_wing')



mov101
mov103
mov104
mov105
mov108
mov109
mov117
mov118
mov119
mov120
mov121
mov123
mov124
mov128
mov129
mov130
mov131
mov132
mov133
mov136
mov137
mov140
mov141
mov143
mov144
mov146
mov148
mov149
mov150
mov152
mov153
mov154
mov155
mov157
mov158
mov163
mov164
mov165
mov166
mov167
mov169
mov172
mov21
mov22
mov3
mov30
mov31
mov35
mov36
mov37
mov38
mov39
mov40
mov41
mov43
mov44
mov47
mov48
mov49
mov5
mov50
mov51
mov53
mov56
mov58
mov59
mov60
mov61
mov62
mov64
mov65
mov66
mov70
mov71
mov72
mov73
mov75
mov76
mov77
mov78
mov8
mov80
mov81
mov83
mov87
mov91
mov92
mov94
mov10
mov101
mov108
mov119
mov128
mov129
mov15
mov19
mov21
mov24
mov28
mov31
mov36
mov40
mov43
mov46
mov48
mov50
mov51
mov52
mov61
mov64
mov67
mov68
mov69
mov7
mov70
mov71
mov72
mov77
mov78
mov95
mov99
mov101
mov103
mov104
mov106
mov107
mov108
mov109
mov110
mov111
mov113
mov24
mov25
mov28
mov31
mov32
mov37
mov41
mov47
mov48
mov49
mov50
mov54
mov55
mov57
mov59
mov62
mov63
mov64
mov66
mov67
mov68
mov69
mov73
mov74
mov76
mov77
mov78